# 要約 
このJupyter Notebook「Qwen2-7b-it」では、言語モデルを用いて「20の質問」ゲームに取り組むエージェントを開発しています。具体的には、質問者と回答者の2つの役割を持つエージェントを作成し、与えられたキーワードをできるだけ少ない質問で推測することを目指しています。

### 取り組んでいる問題
- **「20の質問」ゲーム**: プレイヤーが一つの特定の単語を推測するために、「はい」または「いいえ」で答えられる質問を通じて情報を収集します。このコンペティションでの目標は、エージェントが非常に効率的にターゲットとなる単語を推測することです。

### 使用されている手法とライブラリ
1. **Transformers**: 「Hugging Face」から提供されるTransformersライブラリを使用して、事前学習された言語モデル「Qwen2-7b-instruct」を適用し、トークナイザーおよび因果言語モデルをロードします。
2. **量子化技術**: `BitsAndBytesConfig`を用いて4ビットでのモデルの量子化を行い、メモリ効率を向上させています。これにより、計算リソースの使用が最適化されています。
3. **テキスト生成パイプライン**: テキスト生成のためのパイプラインを設定し、生成されたテキストをベースに質問や推測を行っています。
4. **ロジックの実装**: `Robot`クラスを設計し、質問者、回答者、推測者の3つのモードごとに処理を実行して、ゲームの状態に応じた質問や回答を生成します。
5. **シミュレーション機能**: 提出する前に、シミュレーションを行うためのコードが用意されており、ゲームが実際にプレイされる過程を確認できるようになっています。

このノートブックは、自然言語処理を利用した双方向の対話型AIエージェントを実装するための実用的な例を示しており、特にゲーム理論に基づく戦略的な質問と推測の能力を強調っています。

---


# 用語概説 
以下に、Jupyter Notebookの内容で初心者がつまずきそうな専門用語の簡単な解説を示します。

1. **Causal Language Model (因果言語モデル)**:
   - 出力が生成される際に、与えられた過去の単語に基づいて未来の単語を予測するモデル。ルールとして、常に過去の情報だけを考慮し、その時点での単語に依存して次の単語を生成する。

2. **Quantization (量子化)**:
   - モデルのパラメータの精度を低下させ、より小さなビット数で表現する技術。これにより、計算資源の使用量が減り、モデルの推論速度が向上する。ただし、精度が多少犠牲になることがある。

3. **BitsAndBytesConfig**:
   - Hugginface Transformers内の量子化設定を行うためのクラス。複数の量子化オプション（例えば、ビット数や計算型）を指定し、効率的なモデル読み込みを助ける。

4. **bfloat16**:
   - 16ビットの浮動小数点数で、特に機械学習において計算の効率と精度のバランスをとるために使用される。このフォーマットは、GPUでの計算を高速化しつつ、一定の精度を保つのに有効。

5. **Pipeline**:
   - 機械学習の処理の一連のステップを簡単に実行できるフレームワーク。入力データを特定の処理順序で処理し、モデルの出力を得るための便利な機能。

6. **Meme Efficient SDP**:
   - "Memory Efficient Low-Rank Approximation"の略で、深層学習モデルをメモリ効率よく実装するための技術。特に大規模モデルでのメモリ消費を抑えることを目的とする。

7. **Text Generation (テキスト生成)**:
   - 自然言語処理におけるタスクであり、与えられた入力から意味のある文章を生成すること。次の単語を予測する生成モデルに基づく。

8. **Parse (解析)**:
   - 特定の構文やパターンに基づいてデータを処理し、意味を取り出す行為。ここでの応答解析は、生成されたテキストの中から特定の質問を抽出することを意味する。

9. **Turn (ターン)**:
   - ゲームや対話における一つのラウンドやステップを指す。質問や応答のやり取りが行われる単位。

10. **Asking / Guessing / Answering**:
    - ゲームの中での役割やモードを表す。質問をする「asking」、推測をする「guessing」、そして質問への応答を行う「answering」という三つのモード。

これらの用語は、初心者には馴染みが薄く、実務においてもあまり接触することのない概念であり、理解に苦しむ可能性があります。

---


# Qwen2-7b-it
参考のために、Qwen2-7b-itで作成されたエージェントを共有します。このエージェントは[LLM Leaderboard](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard)で良好な成果を上げています。


In [ ]:
%%bash
mkdir -p /kaggle/working/submission
pip install bitsandbytes accelerate
pip install -U transformers

In [ ]:
%%writefile -a submission/main.py

import os
import torch
import random
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig, pipeline

torch.backends.cuda.enable_mem_efficient_sdp(False)  # CUDAのメモリ効率の良いSDPを無効化

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"  # Kaggleエージェントのパスを指定
if os.path.exists(KAGGLE_AGENT_PATH):  # エージェントのパスが存在する場合
    model_id = os.path.join(KAGGLE_AGENT_PATH, "1")  # パスに基づいてモデルIDを設定
else:
    model_id = "/kaggle/input/qwen2/transformers/qwen2-7b-instruct/1"  # デフォルトのモデルIDを設定


tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)  # トークナイザーをモデルIDに基づいてロード

quantization_config = BitsAndBytesConfig(  # 量子化の設定を行う
    load_in_4bit=True,  # 4ビットでの読み込みを有効にする
    bnb_4bit_quant_type="nf4",  # 4ビット量子化のタイプをnf4に設定
    bnb_4bit_use_double_quant=True,  # 二重量子化を使用する
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算のデータ型をbfloat16に設定
)

model = AutoModelForCausalLM.from_pretrained(  # Causal Language Model（因果言語モデル）をロード
    model_id,
    low_cpu_mem_usage=True,  # CPUメモリ使用量を低くする
    quantization_config=quantization_config,  # 量子化設定を適用
    torch_dtype=torch.float16,  # PyTorchのデータ型をfloat16に設定
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # テキスト生成のためのパイプラインを作成

def parse_response(response):  # 応答を解析する関数
    match = re.search(".+?\?", response.replace('*', ''))  # 応答からクエスチョンを取得
    if match is None:  # マッチが存在しない場合
        question = 'キーワードは「s」で始まりますか？'  # デフォルトの質問を設定
    else:
        question = match.group()  # マッチした質問を取得
    return question

def generate_answer(chat_template):  # 応答を生成する関数
    output = pipe(
        chat_template,  # チャットテンプレートをパイプに渡す
        max_new_tokens=32,  # 最大生成トークン数を32に設定
        do_sample=False,  # サンプリングを無効にする
#         temperature=0.01,  # 温度を設定（コメントアウト）
#         top_p=0.1,  # top-pを設定（コメントアウト）
#         top_k=1,  # top-kを設定（コメントアウト）
        return_full_text=False,  # 完全なテキストを返さない
    )[0]["generated_text"]  # 生成されたテキストを取得
    output = re.sub('<end_of_turn>', '', output)  # テキストから'end_of_turn'を削除
    return output


class Robot:  # Robotクラスを定義
    def __init__(self):
        pass
    
    def on(self, mode, obs):  # モードに基づいて処理を行うメソッド
        assert mode in ["asking", "guessing", "answering"], "modeはasking, answering, guessingのいずれかである必要があります。"  # モードの確認
        if mode == "asking":
            output = self.asker(obs)  # 質問者の場合の処理
        if mode == "answering":
            output = self.answerer(obs)  # 回答者の場合の処理
            if "yes" in output.lower() or "Yes" in output.lower():  # 応答が「yes」の場合
                output = "yes"  # 出力を「yes」に設定
            elif "no" in output.lower() or "No" in output.lower():  # 応答が「no」の場合
                output = "no"  # 出力を「no」に設定
            else:  # その他の場合
                output = "no"  # デフォルトで「no」に設定
        if mode == "guessing":
            output = self.asker(obs)  # 推測者の場合の処理
        return output

    def asker(self, obs):  # 質問者の処理を行うメソッド
        sys_prompt = """
        あなたは「20の質問」ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え出し、質問者のはい/いいえ質問に回答します。
        キーワードは特定の「もの」または「場所」です。
        """
        if obs.turnType =="ask":  # 質問するターン
            ask_prompt = sys_prompt + """
            「20の質問」をプレイしましょう。質問者の役割を果たします。
            はい/いいえの質問をしてください。
            例に従ってください:
            例: キーワードは「モロッコ」
            <assistant: それは都市ですか？
            user: いいえ
            assistant: それは国ですか？
            user: はい
            assistant: アフリカにありますか？
            user: はい
            assistant: mで始まる国名ですか？
            user: はい
            assistant: それはモロッコですか？
            user: はい。>
            """

            chat_template = f"""<start_of_turn>system\n{ask_prompt}\n"""  # チャットテンプレートを作成

            if len(obs.questions)>=1:  # 質問が1つ以上ある場合
                chat_template += "これまでの会話の履歴:\n"  # 会話履歴の追加
                chat_template += "<\n"
                for q, a in zip(obs.questions, obs.answers):  # 質問と応答をループ
                    chat_template += f"assistant:\n{q}\n"
                    chat_template += f"user:\n{a}\n"
                chat_template += ">\n"
                    
            chat_template += """
            ユーザーが選んだ単語について、最初の質問をしてください！
            短く、冗長な表現を避けて、一つの質問だけをしてください.<end_of_turn>
            <start_of_turn>assistant\n
            """
            output = generate_answer(chat_template)  # 応答を生成
            output = parse_response(output)  # 応答を解析
                
        elif obs.turnType == "guess":  # 推測するターン
            conv = ""
            for q, a in zip(obs.questions, obs.answers):  # 質問と応答をループ
                conv += f"""question:\n{q}\nanswer:\n{a}\n"""
            guess_prompt =  sys_prompt + f"""
            あなたの役割は、質問や回答に基づいてキーワードを予測することです。
            例に従ってください:
            例:
            <question: それは都市ですか？
            answer: いいえ
            question: それは国ですか？
            answer: はい
            question: アフリカにありますか？
            answer: はい
            question: mで始まる国名ですか？
            answer: はい
            あなた: モロッコ>
            
            現在のゲームの状態は次の通りです:\n{conv}
            会話に基づいて、キーワードを推測してください。ただ、単語だけを答えて、冗長にしないでください<end_of_turn>
            """
            chat_template = f"""<start_of_turn>system\n{guess_prompt}\n"""  # チャットテンプレートを作成
            chat_template += "<start_of_turn>you:\n"
            output = generate_answer(chat_template)  # 応答を生成
        return output
    
    def answerer(self, obs):  # 回答者の処理を行うメソッド
        ask_prompt = f"""
        あなたは「20の質問」ゲームをプレイするために設計されたAIアシスタントです。 
        このゲームでは、回答者がキーワードを考え出し、質問者のはい/いいえ質問に回答します。
        例に従ってください:
        例: キーワードは「モロッコ」
        <user: それは場所ですか？
        あなた: はい
        user: ヨーロッパにありますか？
        あなた: いいえ
        user: アフリカにありますか？
        あなた: はい
        user: mで始まる国名ですか？
        あなた: はい
        user: それはモロッコですか？
        あなた: はい。>
        "{obs.keyword}"について、次の質問に「はい」または「いいえ」で答えてください:
        {obs.questions[-1]}
        """
        chat_template = f"""<start_of_turn>system\n{ask_prompt}<end_of_turn>\n"""  # チャットテンプレートを作成
        chat_template += "<start_of_turn>Assistant\n"
        output = generate_answer(chat_template)  # 応答を生成
        return output

robot = Robot()  # Robotクラスのインスタンスを作成

def agent(obs, cfg):  # エージェント関数
    if obs.turnType =="ask":  # 質問するターン
        response = robot.on(mode = "asking", obs = obs)
        
    elif obs.turnType =="guess":  # 推測するターン
        response = robot.on(mode = "guessing", obs = obs)
        
    elif obs.turnType =="answer":  # 回答するターン
        response = robot.on(mode = "answering", obs = obs)
        
    if response == None or len(response)<=1:  # 応答が無効な場合
        response = "yes"  # デフォルトの応答を「yes」に設定
        
    return response

In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/input/qwen2/transformers/qwen2-7b-instruct . -C /kaggle/working/submission .

In [ ]:
# # tar.gzファイルの中身を確認するためのコード

# import tarfile
# tar = tarfile.open("/kaggle/working/submission.tar.gz")
# for file in tar.getmembers():
#     print(file.name)

# シミュレーション
エージェントを提出する前にシミュレーションを行いたい場合は、以下のコメントを解除して実行してください。


In [ ]:
# !wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

In [ ]:
# import json
# import pandas as pd
# from submission.main import agent
# from keywords_local import KEYWORDS_JSON

# class Observation:
#     def __init__(self):
#         self.step = 0  # ステップ番号を初期化
#         self.role = "guesser"  # 役割を「推測者」に設定
#         self.turnType = "ask"  # ターンタイプを「質問」に設定
#         self.keyword = "Japan"  # キーワードを「日本」に設定
#         self.category = "country"  # カテゴリを「国」に設定
#         self.questions = []  # 質問リストを初期化
#         self.answers = []  # 応答リストを初期化
#         self.guesses = []  # 推測リストを初期化
        
# def create_keyword_df(KEYWORDS_JSON):  # キーワードデータフレームを作成する関数
#     json_data = json.loads(KEYWORDS_JSON)  # JSONデータをロード

#     keyword_list = []  # キーワードリストを初期化
#     category_list = []  # カテゴリリストを初期化
#     alts_list = []  # 代替リストを初期化

#     for i in range(len(json_data)):  # JSONデータをループ
#         for j in range(len(json_data[i]['words'])):  # 各キーワードをループ
#             keyword = json_data[i]['words'][j]['keyword']  # キーワードを取得
#             keyword_list.append(keyword)  # キーワードをリストに追加
#             category_list.append(json_data[i]['category'])  # カテゴリをリストに追加
#             alts_list.append(json_data[i]['words'][j]['alts'])  # 代替をリストに追加

#     data_pd = pd.DataFrame(columns=['keyword', 'category', 'alts'])  # データフレームを作成
#     data_pd['keyword'] = keyword_list  # キーワード列を設定
#     data_pd['category'] = category_list  # カテゴリ列を設定
#     data_pd['alts'] = alts_list  # 代替列を設定
    
#     return data_pd
    
# keywords_df = create_keyword_df(KEYWORDS_JSON)  # キーワードデータフレームを作成

In [ ]:
# obs = Observation()  # 観察インスタンスを作成
# cfg = "_"

# sample_df = keywords_df.sample()  # キーワードデータフレームからサンプルを取得
# obs.keyword = sample_df["keyword"].values[0]  # サンプルからキーワードを設定
# obs.category = sample_df["category"].values[0]  # サンプルからカテゴリを設定
# alts_list = sample_df["alts"].values[0]  # サンプルから代替リストを設定
# alts_list.append(obs.keyword)  # キーワードを代替リストに追加

# print(f"キーワード: {obs.keyword}")  # キーワードを表示

# for round in range(20):  # 20ラウンドの繰り返し
#     obs.step = round+1  # ステップを更新
    
#     obs.role = "guesser"  # 役割を「推測者」に設定
#     obs.turnType = "ask"  # ターンタイプを「質問」に設定
#     question = agent(obs, cfg)  # 質問を生成
#     obs.questions.append(question)  # 質問をリストに追加
    
#     obs.role = "answerer"  # 役割を「回答者」に設定
#     obs.turnType = "answer"  # ターンタイプを「回答」に設定
#     answer = agent(obs, cfg)  # 応答を生成
#     obs.answers.append(answer)  # 応答をリストに追加
    
#     obs.role = "guesser"  # 役割を「推測者」に設定
#     obs.turnType = "guess"  # ターンタイプを「推測」に設定
#     guess = agent(obs, cfg)  # 推測を生成
#     obs.guesses.append(guess)  # 推測をリストに追加
    
#     print(f"ラウンド: {round+1}")  # ラウンド数を表示
#     print(f"質問: {question}")  # 質問を表示
#     print(f"応答: {answer}")  # 応答を表示
#     print(f"推測: {guess}")  # 推測を表示
    
#     if guess in alts_list:  # 推測が代替リストに存在する場合
#         print("勝利!!")  # 勝利メッセージを表示
#         break